In [1]:
import json
import numpy as np
import pandas as pd
from openpyxl import Workbook

In [18]:
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
data = pd.read_excel('Ground_truth_V2.xlsx')
data = pd.DataFrame(data)
data = data.fillna("")

In [4]:
for i in range(len(data)):
    count_Y = 0
    for k in ["Discovery","Spacy","Html"]:
        if (data.loc[i,k] != ""):
            for j in ["P_"+k,"J_"+k,"S_"+k]:
                if (data.loc[i,j] == "Y"):
                    count_Y+=1
            if (count_Y >=2):
                if (len(data.loc[i,"ground truth"])==0):
                    data.loc[i,"ground truth"] += data.loc[i,k]
                else:
                    data.loc[i,"ground truth"] += "/"+data.loc[i,k]
        count_Y = 0

In [5]:
for i in range(len(data)):
    
    lists = data.loc[i,"ground truth"].split("/")
    
    for k in ["Discovery","Spacy","Html"]:
        
        if(len(data.loc[i,"ground truth"])==0 and len(data.loc[i,k])!=0):
            data.loc[i,k+"_vote"] = "TN"
        
        if(len(data.loc[i,"ground truth"])!=0 and len(data.loc[i,k])==0):
            data.loc[i,k+"_vote"] = "FN"
            
        if (data.loc[i,k] in lists and len(data.loc[i,k])!=0 and len(data.loc[i,"ground truth"])!=0):
            data.loc[i,k+"_vote"] = "TP"
        
        if (data.loc[i,k] not in lists and len(data.loc[i,k])!=0 and len(data.loc[i,"ground truth"])!=0):
            data.loc[i,k+"_vote"] = "FP"

In [6]:
data["Discovery_vote"].value_counts()

FN    1237
TP     858
       590
TN     133
FP      30
Name: Discovery_vote, dtype: int64

In [7]:
Precision = 858/(858+30)
Recall = 858/(858+1237)

Stats_disc = {"Precision": Precision,"Recall":Recall}

In [8]:
data["Spacy_vote"].value_counts()

TP    1507
TN     611
FN     520
       112
FP      98
Name: Spacy_vote, dtype: int64

In [9]:
Precision = 1507/(1507+98)
Recall = 1507/(1507+520)

Stats_spac = {"Precision": Precision,"Recall":Recall}

In [10]:
data["Html_vote"].value_counts()

FN    1312
TP     799
       697
TN      26
FP      14
Name: Html_vote, dtype: int64

In [11]:
Precision = 799/(799+14)
Recall = 799/(799+1312)

Stats_Html = {"Precision": Precision,"Recall":Recall}

In [15]:
print("Discovery")
print(Stats_disc)
print("--"*50)
print("Spacy")
print(Stats_spac)
print("--"*50)
print("HTML")
print(Stats_Html)
print("--"*50)

Discovery
{'Precision': 0.9662162162162162, 'Recall': 0.40954653937947494}
----------------------------------------------------------------------------------------------------
Spacy
{'Precision': 0.9389408099688473, 'Recall': 0.7434632461766156}
----------------------------------------------------------------------------------------------------
HTML
{'Precision': 0.982779827798278, 'Recall': 0.3784936049265751}
----------------------------------------------------------------------------------------------------


In [45]:
data

,Entity,ground truth,Discovery,Discovery_vote,P_Discovery,J_Discovery,S_Discovery,Spacy,Spacy_vote,P_Spacy,J_Spacy,S_Spacy,Html,Html_vote,P_Html,J_Html,S_Html
0,an eight year span,,,,,,,DATE,TN,,,,,,,,
1,NBA Finals,,,,,,,ORG,TN,,,,,,,,
2,third,ORDINAL,,FN,,,,ORDINAL,TP,Y,Y,Y,,FN,,,
3,All-NBA,,,,,,,PRODUCT,TN,,,,,,,,
4,ESPN's,org,,FN,,,,,FN,,,,org,TP,Y,Y,Y
5,181,CARDINAL,,FN,,,,CARDINAL,TP,Y,Y,Y,,FN,,,
6,Washington Wizards,Organization/ORG/org,Organization,TP,Y,Y,Y,ORG,TP,Y,Y,Y,org,TP,Y,Y,Y
7,2006,DATE,,FN,,,,DATE,TP,Y,,Y,,FN,,,
8,Spurs,Organization/ORG,Organization,TP,Y,Y,Y,ORG,TP,Y,Y,Y,,FN,,,
9,the regular season,,,,,,,DATE,TN,,,,,,,,


# Visualization

In [14]:
data["ground truth"]

0                   
1                   
2            ORDINAL
3                   
4                org
            ...     
2843                
2844    LOC/location
2845                
2846                
2847        Quantity
Name: ground truth, Length: 2848, dtype: object

In [16]:
entity_type=['Location',
 'Person',
 'Organization',
 'JobTitle',
 'Company',
 'Quantity',
 'Facility',
 'GeographicFeature',
 'EmailAddress',
 'PrintMedia',
 'HealthCondition',
 'Crime',
 'Sport',
 'Drug',
 'TwitterHandle',
 'Hashtag',
 'Broadcaster',
 'Movie',
 'Vehicle',
 'Award',
 'Anatomy',
 'NaturalEvent',
 'TelevisionShow',
 'SportingEvent',
 'MusicGroup',
 'IPAddress']

In [33]:
entity_type_2=['CARDINAL',
 'ORG',
 'DATE',
 'FAC',
 'PERSON',
 'WORK_OF_ART',
 'LOC',
 'ORDINAL',
 'GPE',
 'NORP',
 'PERCENT',
 'QUANTITY',
 'PRODUCT',
 'MONEY',
 'EVENT',
 'TIME',
 'LAW',
 'LANGUAGE']

In [34]:
entity_type_3=['person', 'org', 'location', 'chron', 'money', 'a']

In [19]:
a={}
for i in range(len(data)):
    lists = data.loc[i,"ground truth"].split("/")
    for item in lists:
        if item in a:
            a[item]+=1
        else:
            a[item]=1

In [211]:
for entity in [entity_type,entity_type_2,entity_type_3]:
    l=[]
    for i in entity:
        if i in a:
            l.append(a[i])
        else:
            l.append(0)
#     plt.figure(figsize=(12,8))
#     sn.barplot(y=entity,x=l,orient=0)
    for p,q in zip(entity,l):
        print(p,q)
    print("-"*50)
#     plt.show()

Location 177
Person 236
Organization 145
JobTitle 46
Company 31
Quantity 128
Facility 57
GeographicFeature 8
EmailAddress 2
PrintMedia 8
HealthCondition 4
Crime 3
Sport 10
Drug 0
TwitterHandle 1
Hashtag 1
Broadcaster 3
Movie 0
Vehicle 0
Award 0
Anatomy 0
NaturalEvent 0
TelevisionShow 1
SportingEvent 0
MusicGroup 0
IPAddress 0
--------------------------------------------------
CARDINAL 258
ORG 255
DATE 180
FAC 24
PERSON 281
WORK_OF_ART 12
LOC 17
ORDINAL 36
GPE 199
NORP 34
PERCENT 39
QUANTITY 114
PRODUCT 5
MONEY 30
EVENT 11
TIME 45
LAW 6
LANGUAGE 6
--------------------------------------------------
person 205
org 182
location 252
chron 89
money 51
a 20
--------------------------------------------------


In [47]:
b={}
for i in a:
    if a[i]>25:
        b[i]=a[i]

In [48]:
b

{'': 723,
 'ORDINAL': 36,
 'org': 182,
 'CARDINAL': 258,
 'Organization': 145,
 'ORG': 255,
 'DATE': 180,
 'PERCENT': 39,
 'Person': 236,
 'PERSON': 281,
 'person': 205,
 'Location': 177,
 'GPE': 199,
 'location': 252,
 'Quantity': 128,
 'NORP': 34,
 'QUANTITY': 114,
 'JobTitle': 46,
 'Facility': 57,
 'chron': 89,
 'money': 51,
 'MONEY': 30,
 'TIME': 45,
 'Company': 31}

# Person

In [134]:
Htm={}
for i in list(b.keys()):
    if i in entity_type_3:
        Htm[i]=0

In [137]:
def rec_pre(d):
    a={}
    p=d["TP"]/(d["TP"]+d["FP"])
    r=d["TP"]/(d["TP"]+d["FN"])
    a["Precision"]=p
    a["Recall"]=r
    return(a)    

In [208]:
d={"TP":0,"TN":0,"FP":0,"FN":0}
for i in range(len(data)):
    lis = data.loc[i][1].split("/")
    if ("money" in data.loc[i][12] or ("Quantity" in lis)or ("MONEY" in lis)):
        x=data.loc[i][13]
        if x in ["TP","TN","FP","FN"]:
            d[x]+=1
Htm["money"]=rec_pre(d)

In [209]:
Htm

{'org': {'Precision': 0.9639175257731959, 'Recall': 0.4594594594594595},
 'person': {'Precision': 0.981042654028436, 'Recall': 0.5594594594594594},
 'location': {'Precision': 0.9695817490494296, 'Recall': 0.6343283582089553},
 'chron': {'Precision': 1.0, 'Recall': 0.356},
 'money': {'Precision': 1.0, 'Recall': 0.31097560975609756}}

In [194]:
Spa

{'ORDINAL': {'Precision': 1.0, 'Recall': 1.0},
 'CARDINAL': {'Precision': 0.9922480620155039, 'Recall': 1.0},
 'ORG': {'Precision': 0.82, 'Recall': 0.6373056994818653},
 'DATE': {'Precision': 0.9494949494949495, 'Recall': 0.9082125603864735},
 'PERCENT': {'Precision': 0.9523809523809523, 'Recall': 0.5673758865248227},
 'PERSON': {'Precision': 0.8307692307692308, 'Recall': 0.7848837209302325},
 'GPE': {'Precision': 0.8143939393939394, 'Recall': 0.671875},
 'NORP': {'Precision': 0.9142857142857143, 'Recall': 1.0},
 'QUANTITY': {'Precision': 0.9738562091503268, 'Recall': 0.7095238095238096},
 'MONEY': {'Precision': 0.9294117647058824, 'Recall': 0.5},
 'TIME': {'Precision': 0.9439252336448598, 'Recall': 0.8416666666666667}}

In [162]:
Dis

{'Organization': {'Precision': 0.9521276595744681,
  'Recall': 0.4601542416452442},
 'Person': {'Precision': 0.9673469387755103, 'Recall': 0.6475409836065574},
 'Location': {'Precision': 0.9267241379310345, 'Recall': 0.587431693989071},
 'Quantity': {'Precision': 0.9846153846153847, 'Recall': 0.512},
 'JobTitle': {'Precision': 1.0, 'Recall': 1.0},
 'Facility': {'Precision': 0.9388646288209607, 'Recall': 0.587431693989071},
 'Company': {'Precision': 0.910828025477707, 'Recall': 0.40509915014164305}}

In [212]:
entity_type

['Location',
 'Person',
 'Organization',
 'JobTitle',
 'Company',
 'Quantity',
 'Facility',
 'GeographicFeature',
 'EmailAddress',
 'PrintMedia',
 'HealthCondition',
 'Crime',
 'Sport',
 'Drug',
 'TwitterHandle',
 'Hashtag',
 'Broadcaster',
 'Movie',
 'Vehicle',
 'Award',
 'Anatomy',
 'NaturalEvent',
 'TelevisionShow',
 'SportingEvent',
 'MusicGroup',
 'IPAddress']

# Explore


In [269]:
df_di = pd.read_excel("today.xlsx",sheet_name="Sheet2")

In [270]:
df_c=df_di[["P","J","S"]]

In [271]:
df_c.fillna(1,inplace=True)

c:\users\jayesh\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [275]:
df_c["TP"] = df_c.sum(axis=1)

c:\users\jayesh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [278]:
df_c["TP"]=df_c["TP"].replace(3.0,1)

c:\users\jayesh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [279]:
df_di["TP"]=df_c["TP"]

In [280]:
df_di

,Entities,Examples,P,J,S,TP,FP
0,Person,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,George Floyd,NaN,NaN,NaN,1.0,NaN
2,NaN,Derek Chauvin,NaN,NaN,NaN,1.0,NaN
3,NaN,Donald Trump,NaN,NaN,NaN,1.0,NaN
4,NaN,coronavirus,0.0,0.0,0.0,0.0,NaN
5,NaN,President Donald Trump,NaN,NaN,NaN,1.0,NaN
6,NaN,President Trump,NaN,NaN,NaN,1.0,NaN
7,NaN,Breonna Taylor,NaN,NaN,NaN,1.0,NaN
8,NaN,George,NaN,NaN,NaN,1.0,NaN
9,NaN,John,NaN,NaN,NaN,1.0,NaN


In [281]:
df_di["Entities"]=df_di["Entities"].fillna(method='ffill')

In [284]:
df_stats=df_di.groupby("Entities").sum()

In [285]:
df_stats["TP"]=df_stats["TP"]-1

In [289]:
df_stats["TP+FP"]=df_di.groupby("Entities").count()["Examples"]

In [293]:
df_stats.drop(["P","J","S"],axis=1,inplace=True)

In [295]:
df_stats["FP"] = df_stats["TP+FP"]-df_stats["TP"]

In [299]:
df_stats["Precision"] = df_stats["TP"]*100/df_stats["TP+FP"]

In [300]:
df_stats

,TP,FP,TP+FP,Precision
Entities,,,,
Anatomy,100.0,0.0,100,100.000000
Award,96.0,4.0,100,96.000000
Broadcaster,97.0,3.0,100,97.000000
Company,90.0,10.0,100,90.000000
Crime,97.0,1.0,98,98.979592
Drug,94.0,6.0,100,94.000000
EmailAddress,100.0,0.0,100,100.000000
Facility,94.0,6.0,100,94.000000
GeographicFeature,92.0,8.0,100,92.000000
